In [1]:
import os
import numpy as np
import networkx as nx

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_aer.primitives import Sampler
from qiskit_aer.backends import AerSimulator
from qiskit.circuit.library import TwoLocal
#from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit import transpile, QuantumCircuit
from qiskit_optimization.translators import to_docplex_mp
from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader
from qiskit_optimization.translators import from_docplex_mp

### Create Problem Hamiltonian 

In [2]:
def adjacency_to_Zop(qubo_array, constant):

    """

    Function that takes the adjacency matrix for the qubo and converts it to a Pauli Z operator

    Directly consrtructs the quadractic program with reference to this page
    """
    number_of_variables = len(qubo_array[1]) # gets the number of variables from the length of the square qubo matrix
    #mdl = Model('model_name')
    mod = QuadraticProgram()

    for variable in range(0,number_of_variables): # creates the binary variables from the size of the matrix 
        var_name = "x_" +str(variable)
        mod.binary_var(name=var_name)

    mod.minimize(constant = 2,quadratic = qubo_array)  # can put in all constraints as quadractic as the binary variables mean that x_0 ^ 2 = x_0 in both cases 
                                                       #  not sure of the impact of this on performance however 
    
    #qp = from_docplex_mp(mod)
    qp2qubo = QuadraticProgramToQubo()
    qubo = qp2qubo.convert(mod)
    quboOp, offset = qubo.to_ising()
    
    return {'Op': quboOp, 'offset': offset}

In [3]:
file_path = '/Users/vaibhawkumar/vaibhaw/projects/algos/Client-Moderna-Optimization/vqe/sequential_VQE/MS_9_80.npy'

In [4]:
with open(file_path,'rb') as f:
    A = np.load(f).T
    b = np.load(f)
#print(qubo.prettyprint())

num_qubits = A.shape[1]
scaling = np.max(b)

adj = A.T@A -A.T@b - b.T@A  
offset = b.T@b

In [5]:
Hp = adjacency_to_Zop(adj, constant=offset)

### IQP circuit

#### the circuit layout we use will be a 4D hyper cube: See this https://arxiv.org/pdf/2402.03211

In [6]:
n_qubits = 2 ** 7
G_hp_cube = nx.generators.hypercube_graph(n=7) #
G_hp_cube.edges()

def tuple_to_int(binary_tuple):

    binary_string = "".join(map(str, binary_tuple))  

    return int(binary_string, 2)

edge_list = []
for elem in G_hp_cube.edges():
    edge_list.append((tuple_to_int(elem[0]), tuple_to_int(elem[1])))

G_layout = nx.Graph()
G_layout.add_nodes_from(range(0, n_qubits))
for edge in edge_list:
    G_layout.add_edge(edge[0], edge[1])

#nx.draw(G_layout, with_labels=True, font_weight='bold')

import iqpopt as iqp
from iqpopt.utils import local_gates, nearest_neighbour_gates

n_qubits = 2**7
gates = nearest_neighbour_gates(G_layout, distance=1, max_weight=2) 

circuit = iqp.IqpSimulator(n_qubits, gates)

#### Let's find the transpiled depth of IQP circuits

In [8]:
qc = QuantumCircuit(n_qubits)
for i in range(0, n_qubits):
    qc.h(i)
    
for edge in edge_list:
    qc.rzz(np.pi/2, edge[0], edge[1])
    
for i in range(0, n_qubits):
    qc.h(i)

from qiskit.transpiler import PassManager
from qiskit.circuit.library import EfficientSU2
from qiskit_ibm_transpiler.ai.routing import AIRouting
 
ai_passmanager = PassManager([
  AIRouting(backend_name="ibm_torino", optimization_level=3, layout_mode="optimize", local_mode=True)
])

for i in range(0, 5):
    qc_trans = ai_passmanager.run(qc)
    print(qc_trans.depth(lambda x:x.operation.num_qubits==2))

In [11]:
## we create a binary array representing the Hamiltonian, needed for IQPOpt

def find(s, ch):
    return tuple(i for i, ltr in enumerate(s) if ltr == ch)

ising_adj = {}
for op in Hp['Op'].label_iter():
    ising_adj[find(op[0], 'Z')] = op[1]
    
binary_arr = []
for key in ising_adj.keys():
    bin_str = n_qubits * [0] #a bit string with n_qubit zeros
    for item in key:
        bin_str[n_qubits-item-1] = 1 #to account for endian convention in qiskit
    binary_arr.append(bin_str)

### Compute expectation

In [12]:
import jax
import jax.numpy as jnp

ops = jnp.array(binary_arr) #binary array representing operators
params = jnp.ones(len(circuit.gates))
n_samples = 1000
key = jax.random.PRNGKey(42)

expval, std = circuit.op_expval(params, op, n_samples, key)

AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
expval

### Training

In [13]:
def loss(params, circuit, ops, n_samples, key):
    expvals = circuit.op_expval(params, ops, n_samples, key)[0] 
    return jnp.sum(jnp.array([coeff.real * exp for coeff, exp in zip(ising_adj.values(), expvals)])) + offset

In [14]:
import numpy as np
import matplotlib.pyplot as plt

optimizer = "Adam" 
stepsize = 0.001
n_iters = 1000
params_init = np.random.normal(0, 1/np.sqrt(n_qubits), len(circuit.gates))
ops = ops
n_samples = 2 ** 13

loss_kwargs = {
    "params": params_init,
    "circuit": circuit,
    "ops": ops,
    "n_samples": n_samples,
}

trainer = iqp.Trainer(optimizer, loss, stepsize)
trainer.train(n_iters, loss_kwargs)

params = trainer.final_params
plt.plot(trainer.losses)

Training Progress:  50%|██████████████████████████████████████████▋                                           | 496/1000 [17:24:11<17:41:02, 126.31s/it, loss=90404468.463107, elapsed time=126, total time=6.26e+4]
Exception ignored in: <function _xla_gc_callback at 0x315a9f010>
Traceback (most recent call last):
  File "/Users/vaibhawkumar/miniconda3/envs/cplex-qiskit/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
KeyboardInterrupt: 


KeyboardInterrupt: 